# MongoDB Atlas Indexes


<br>




In [1]:
import pymongo
import os
from pprint import pprint as pp
from pymongo import MongoClient

password = os.environ['PASSWORD']

client = MongoClient('mongodb+srv://pepe:' + password +'@cluster0-dajh0.mongodb.net/test?retryWrites=true&w=majority')
pp(client.server_info())

{'$clusterTime': {'clusterTime': Timestamp(1585910982, 1),
                  'signature': {'hash': b'\t\xcd\x0b\xa8\xff\xca\x9a\xdb'
                                        b'\xcc\xde\x91\t\x04]\xb6\x9a'
                                        b'\x0e\xd9\xbb\xd9',
                                'keyId': 6811077507295477762}},
 'allocator': 'tcmalloc',
 'bits': 64,
 'buildEnvironment': {'cc': '/opt/mongodbtoolchain/v3/bin/gcc: gcc (GCC) 8.2.0',
                      'ccflags': '-fno-omit-frame-pointer -fno-strict-aliasing '
                                 '-ggdb -pthread -Wall -Wsign-compare '
                                 '-Wno-unknown-pragmas -Winvalid-pch -Werror '
                                 '-O2 -Wno-unused-local-typedefs '
                                 '-Wno-unused-function '
                                 '-Wno-deprecated-declarations '
                                 '-Wno-unused-const-variable '
                                 '-Wno-unused-but-set-variable '
  

In [2]:
database_names = client.database_names()
pp(database_names)

['admin',
 'config',
 'local',
 'sample_airbnb',
 'sample_analytics',
 'sample_geospatial',
 'sample_mflix',
 'sample_supplies',
 'sample_training',
 'sample_weatherdata']


In [3]:
db = client.sample_airbnb
collection_names = db.collection_names()
pp(collection_names)

['listingsAndReviews']


Getting index information

In [5]:
collection = db.listingsAndReviews
for i in dir(collection):
    if 'index' in i:
        pp(i)

'_Collection__create_index'
'create_index'
'create_indexes'
'drop_index'
'drop_indexes'
'ensure_index'
'index_information'
'list_indexes'
'reindex'


## Index Information



In [7]:

index_information = collection.index_information()#['bedrooms_1_address.country_1']
pp(index_information)


{'_id_': {'key': [('_id', 1)],
          'ns': 'sample_airbnb.listingsAndReviews',
          'v': 2},
 'address.location_2dsphere': {'2dsphereIndexVersion': 3,
                               'background': True,
                               'key': [('address.location', '2dsphere')],
                               'ns': 'sample_airbnb.listingsAndReviews',
                               'v': 2},
 'name_1': {'background': True,
            'key': [('name', 1)],
            'ns': 'sample_airbnb.listingsAndReviews',
            'v': 2},
 'property_type_1_room_type_1_beds_1': {'background': True,
                                        'key': [('property_type', 1),
                                                ('room_type', 1),
                                                ('beds', 1)],
                                        'ns': 'sample_airbnb.listingsAndReviews',
                                        'v': 2}}


In [8]:
# iterate the dict object returned by the method call 
for key, val in index_information.items(): 
    print(key, "--", val)

_id_ -- {'v': 2, 'key': [('_id', 1)], 'ns': 'sample_airbnb.listingsAndReviews'}
property_type_1_room_type_1_beds_1 -- {'v': 2, 'key': [('property_type', 1), ('room_type', 1), ('beds', 1)], 'background': True, 'ns': 'sample_airbnb.listingsAndReviews'}
name_1 -- {'v': 2, 'key': [('name', 1)], 'background': True, 'ns': 'sample_airbnb.listingsAndReviews'}
address.location_2dsphere -- {'v': 2, 'key': [('address.location', '2dsphere')], 'background': True, '2dsphereIndexVersion': 3, 'ns': 'sample_airbnb.listingsAndReviews'}


## List indexes




In [10]:
index_cursor = collection.list_indexes() 
# iterate the pymongo.command_cursor.CommandCursor object 
for index in index_cursor: 
# print the bson.son.SON object for each iteration 
    print ("\n") 
    print ("index.keys():", index.keys()) 
    print ("NAME:", index["name"]) # index name 
    print ("VERSION:", index["v"]) # index version 

# iterate the index bson object's key-value pairs 
# using the object's items() method 
for key, val in index.items(): 
    print (key, "--->", val, "-- type:", type(val))



index.keys(): ['v', 'key', 'name', 'ns']
NAME: _id_
VERSION: 2


index.keys(): ['v', 'key', 'name', 'background', 'ns']
NAME: property_type_1_room_type_1_beds_1
VERSION: 2


index.keys(): ['v', 'key', 'name', 'background', 'ns']
NAME: name_1
VERSION: 2


index.keys(): ['v', 'key', 'name', 'background', '2dsphereIndexVersion', 'ns']
NAME: address.location_2dsphere
VERSION: 2
v ---> 2 -- type: <class 'int'>
key ---> SON([('address.location', '2dsphere')]) -- type: <class 'bson.son.SON'>
name ---> address.location_2dsphere -- type: <class 'str'>
background ---> True -- type: <class 'bool'>
2dsphereIndexVersion ---> 3 -- type: <class 'int'>
ns ---> sample_airbnb.listingsAndReviews -- type: <class 'str'>


## Execution Stats




In [12]:
collection.find({})
query = collection.find({'bedrooms': 1, 'beds' : 1, 'address.country' : 'Brazil'})
for i in query.limit(2):
    pp(i)

{'_id': '10030955',
 'access': '',
 'accommodates': 2,
 'address': {'country': 'Brazil',
             'country_code': 'BR',
             'government_area': 'Lagoa',
             'location': {'coordinates': [-43.205047082633435,
                                          -22.971950988341874],
                          'is_location_exact': True,
                          'type': 'Point'},
             'market': 'Rio De Janeiro',
             'street': 'Rio de Janeiro, Rio de Janeiro, Brazil',
             'suburb': 'Lagoa'},
 'amenities': ['TV',
               'Cable TV',
               'Internet',
               'Wifi',
               'Air conditioning',
               'Pool',
               'Kitchen',
               'Free parking on premises',
               'Doorman',
               'Gym',
               'Elevator',
               'Buzzer/wireless intercom',
               'Family/kid friendly',
               'Washer',
               'Essentials',
               '24-hour check-in'],
 

In [13]:
execution_stats = query.explain()['queryPlanner']
pp(execution_stats)

{'indexFilterSet': False,
 'namespace': 'sample_airbnb.listingsAndReviews',
 'parsedQuery': {'$and': [{'address.country': {'$eq': 'Brazil'}},
                          {'bedrooms': {'$eq': 1}},
                          {'beds': {'$eq': 1}}]},
 'plannerVersion': 1,
 'rejectedPlans': [],
 'winningPlan': {'inputStage': {'direction': 'forward',
                                'filter': {'$and': [{'address.country': {'$eq': 'Brazil'}},
                                                    {'bedrooms': {'$eq': 1}},
                                                    {'beds': {'$eq': 1}}]},
                                'stage': 'COLLSCAN'},
                 'limitAmount': 2,
                 'stage': 'LIMIT'}}


2020-04-02T19:29:01.086+0000 I COMMAND [conn1190] command sample_airbnb.listingsAndReviews command: find { find: "listingsAndReviews", filter: { bedrooms: 3, address: { country: "Portugal" } }, limit: 2, lsid: { id: UUID("83394815-c4e6-4152-a9d9-a92d8bbd814c") }, $clusterTime: { clusterTime: Timestamp(1585855734, 19), signature: { hash: BinData(0, AF7BF4B36B6D32BF28E9448E2391B6BF8570BFF8), keyId: 6811077507295477762 } }, $db: "sample_airbnb", $readPreference: { mode: "primary" } } planSummary: COLLSCAN keysExamined:0 docsExamined:5555 cursorExhausted:1 numYields:44 nreturned:0 queryHash:9CDD0B83 planCacheKey:9CDD0B83 reslen:246 locks:{ ReplicationStateTransition: { acquireCount: { w: 45 } }, Global: { acquireCount: { r: 45 } }, Database: { acquireCount: { r: 45 } }, Collection: { acquireCount: { r: 45 } }, Mutex: { acquireCount: { r: 1 } } } storage:{ data: { bytesRead: 94480432, timeReadingMicros: 284341 } } protocol:op_msg 298ms<br>




## Create Index



In [15]:
resp = collection.create_index([("bedrooms", 1),("address.country", 1)])

print ("index response:", resp)

index response: bedrooms_1_address.country_1


In [16]:
execution_stats = query.explain()['queryPlanner']
pp(execution_stats)

{'indexFilterSet': False,
 'namespace': 'sample_airbnb.listingsAndReviews',
 'parsedQuery': {'$and': [{'address.country': {'$eq': 'Brazil'}},
                          {'bedrooms': {'$eq': 1}},
                          {'beds': {'$eq': 1}}]},
 'plannerVersion': 1,
 'rejectedPlans': [],
 'winningPlan': {'inputStage': {'filter': {'beds': {'$eq': 1}},
                                'inputStage': {'direction': 'forward',
                                               'indexBounds': {'address.country': ['["Brazil", '
                                                                                   '"Brazil"]'],
                                                               'bedrooms': ['[1, '
                                                                            '1]']},
                                               'indexName': 'bedrooms_1_address.country_1',
                                               'indexVersion': 2,
                                               'isMultiKey

In [17]:
execution_stats = collection.find({'bedrooms': 1}).explain()['queryPlanner']
pp(execution_stats)

{'indexFilterSet': False,
 'namespace': 'sample_airbnb.listingsAndReviews',
 'parsedQuery': {'bedrooms': {'$eq': 1}},
 'plannerVersion': 1,
 'rejectedPlans': [],
 'winningPlan': {'inputStage': {'direction': 'forward',
                                'indexBounds': {'address.country': ['[MinKey, '
                                                                    'MaxKey]'],
                                                'bedrooms': ['[1, 1]']},
                                'indexName': 'bedrooms_1_address.country_1',
                                'indexVersion': 2,
                                'isMultiKey': False,
                                'isPartial': False,
                                'isSparse': False,
                                'isUnique': False,
                                'keyPattern': {'address.country': 1,
                                               'bedrooms': 1},
                                'multiKeyPaths': {'address.country': [],
         

## Drop Index



In [20]:
result = collection.drop_index("bedrooms_1_address.country_1")
pp(result)

None
